## Visualization w/ folium

In [1]:
# plot coordinates using folium, python wrapper of Leaflet (a JavaScript library for interactive maps)
import folium
from folium.plugins import FastMarkerCluster
import pandas as pd
import pkg_resources

In [2]:
modules_used = 'folium,pandas'.split(',')
print('Versions')
for mod in modules_used:
    print('{}: {}'.format(mod, pkg_resources.get_distribution(mod).version))

Versions
folium: 0.6.0
pandas: 0.23.4


In [4]:
data = pd.read_csv('./data/clean_data.csv', converters={'Coordinates':eval},\
                   parse_dates=['Occurred', 'Reported'])
data.head()

,Unnamed: 0,Incident,Comments,Reported,Coordinates,Occurred
0,0,lost,woman reports losing a canon power shot digita...,2010-07-01 12:42:00,"(41.7892236, -87.5980197)",2010-06-28 19:45:00
1,5,lost,staff member reports losing wallet containing ...,2010-07-01 23:09:00,"(41.66861859999999, -87.7841447)",2010-07-02 03:00:00
2,72,lost,patient's cell phone missing after she transfe...,2010-07-18 19:19:00,"(41.7892116, -87.60406619999999)",2010-07-18 19:00:00
3,111,lost,patient reports a hp computer missing after sh...,2010-07-24 19:07:00,"(41.7892116, -87.60406619999999)",2010-07-24 16:00:00
4,122,lost,patient reports loss of eye glasses,2010-07-25 13:41:00,"(41.7892116, -87.60406619999999)",2010-07-23 17:00:00


In [5]:
# add year column and groupby it
data['year'] = data.Reported.dt.year
group_year = data.groupby('year')
data.head()

,Unnamed: 0,Incident,Comments,Reported,Coordinates,Occurred,year
0,0,lost,woman reports losing a canon power shot digita...,2010-07-01 12:42:00,"(41.7892236, -87.5980197)",2010-06-28 19:45:00,2010.0
1,5,lost,staff member reports losing wallet containing ...,2010-07-01 23:09:00,"(41.66861859999999, -87.7841447)",2010-07-02 03:00:00,2010.0
2,72,lost,patient's cell phone missing after she transfe...,2010-07-18 19:19:00,"(41.7892116, -87.60406619999999)",2010-07-18 19:00:00,2010.0
3,111,lost,patient reports a hp computer missing after sh...,2010-07-24 19:07:00,"(41.7892116, -87.60406619999999)",2010-07-24 16:00:00,2010.0
4,122,lost,patient reports loss of eye glasses,2010-07-25 13:41:00,"(41.7892116, -87.60406619999999)",2010-07-23 17:00:00,2010.0


In [6]:
def create_map(df, location, year=None, callback=None):
    
    # initialize map
    map_ = folium.Map(location=location, zoom_start=14, min_zoom=14)
    
    
    if isinstance(year, type(None)):
        grouped = df.groupby('Incident')
        for key in grouped.groups.keys():
            data_ = grouped.get_group(key)['Coordinates']
            FastMarkerCluster(data=data_, callback=callback, name=key).add_to(map_)
            
        # make layers interactive
        folium.LayerControl().add_to(map_)
        return map_
        
    if year in df.year.unique():
        incidents = df.Incident.unique()
        grouped = df.groupby(['year', 'Incident'])

        for val in incidents:
            try:
                data_ = grouped.get_group((year, val))['Coordinates']
            except KeyError:
                # prevents error from years when mental health incidents weren't reported
                continue

            FastMarkerCluster(data=data_, callback=callback, name=val).add_to(map_)
    else:
        return "Enter valid year or leave as None"
    
    folium.LayerControl().add_to(map_)
    return map_

In [7]:
hyde_park = [41.7943, -87.5907]  # location start

# string representation of a valid Javascript function 
callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({icon: "map-marker", markerColor: "red"});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""

In [8]:
crime_map = create_map(data, location=hyde_park, year=2018, callback=callback)
crime_map

In [8]:
# save map (uncomment line)
#crime_map.save('2018_cluster_map.html')